## 모의고사 03회
### 작업형 제 1유형
#### 01.

In [35]:
import pandas as pd
import numpy as np

## 00. 데이터 가져오기
df1 = pd.read_csv('../data/모의고사/03회/economics.csv', encoding = 'cp949')
# print(df1.head())

## 01. 필요한 칼럼만 저장
df1 = df1[ ['데이터수집월', '실업자수_천명'] ]
# print(df1.head())
# print(df1.info())

## 02. 수집월을 파생변수로 만들기
df1[['수집년', '수집월', '수집일']] = df1['데이터수집월'].str.split('-', expand=True)
# print(df1.head())
df1 = df1.drop(columns = ['데이터수집월', '수집월', '수집일'], axis = 1)
# print(df1.head())
# print(df1.info())

# 수집년 값 확인
# print(df1['수집년'].value_counts())
# 공백 제거
df1['수집년'] = df1['수집년'].str.replace(' ', '')
# print(df1['수집년'].value_counts())

## 03. 연도별 실업자수의 월평균
df1_year = df1.groupby('수집년').mean()
# print(df1_year.sort_values('실업자수_천명'))  # 2010이 제일 많음

## 04. 2010년의 분산 (만명단위로 변경하고 소수 둘째자리까지 출력)
df1_2010 = df1[ df1['수집년'] == '2010' ]
# print(df1_2010)

# 단위 변경 >> 만단위면 나누가 10
df1_2010['실업자수_만명'] = df1_2010['실업자수_천명'] / 10
# print(df1_2010)

# 분산 계산
result1 = round(df1_2010['실업자수_만명'].var(), 2)
print(result1)

1111.35


C:\Users\kbr_u\AppData\Local\Temp\ipykernel_17772\4137647741.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2010['실업자수_만명'] = df1_2010['실업자수_천명'] / 10


#### 02.

In [77]:
import pandas as pd
import numpy as np

## 00. 데이터 가져오기
df2 = pd.read_csv('../data/모의고사/03회/Airline.csv', encoding = 'cp949')
# print(df2.info())

## 01. 필요한 칼럼만 저장
df2 = df2[ ['Ages', 'Departure_Delay_in_Minutes', 'Customer_Type'] ]
# print(df2.info())
# print(df2['Ages'].value_counts())
# print(df2['Customer_Type'].value_counts())

## 02. 결측값 확인
# print(df2.isnull().sum())  # 없음

## 03. 연령대 표기 통일

age_mapping = {
    '[40, 50)': '40_49',
    '[50, 60)': '50_59',
    '[10, 20)': '10_19',
    '[20, 30)': '20_29',
    '[30, 40)': '30_39'
}

# 연령대를 xx_xx 형식으로 변경 (매핑이 되지 않는 값은 fillna 함수로 원래 값으로 채우기)
df2['Ages'] = df2['Ages'].map(age_mapping).fillna(df2['Ages'])
# print(df2['Ages'].value_counts())

## 04. 연령대별 지연시간 평균 구하기
df2_age = df2['Departure_Delay_in_Minutes'].groupby(df2['Ages']).mean()
# print(df2_age.sort_values())       # 50_59의 평균이 가장 김

df2_5059 = df2[ df2['Ages'] == '50_59']

## 05. Customer_Type이 L인 경우의 인원수 (정수로)
df2_5059_L = df2_5059[ df2_5059['Customer_Type'] == 'L' ]

result2 = int(len(df2_5059_L))
print(result2)

1101


#### 03. 

In [153]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 00. 데이터 가져오기
df3 = pd.read_csv('../data/모의고사/03회/spb_daily.csv', encoding = 'cp949')
# print(df3.shape)  # (2619, 6)
# print(df3.info())

## 01. 필요한 칼럼만 저장
df3 = df3[ ['대여일시', '이용시간', '대여소번호', '이용거리'] ]
# print(df3.head())

## 02. 대여일시 칼럼에서 시간 정보만 저장
df3[['대여일', '시각']] = df3['대여일시'].str.split(' ', expand=True)
df3 = df3.drop(columns=['대여일', '대여일시'], axis=1)
df3['시각'] = df3['시각'].str[:2]
# print(df3['시각'].value_counts())

## 03. 시간대별 평균 이용시간 구하기
# 이용시간 칼럼을 int형으로 변경
df3['이용시간'] = pd.to_numeric(df3['이용시간'], errors='coerce')

df3_bytime = df3.groupby('시각')['이용시간'].mean()
# print(df3_bytime.sort_values())  # 11시가 가장 큼

## 04. 11시의 대여지점별 누적 이용거리 계산 (400번대만)
# print(df3['대여소번호'].value_counts())

# 대여소번호의 공백 제거
df3['대여소번호'] = df3['대여소번호'].str.replace(' ', '')

# 숫자형으로 변환
df3['대여소번호'] = pd.to_numeric(df3['대여소번호'])
df3['이용거리'] = pd.to_numeric(df3['이용거리'])

# 400번대만 남기고 삭제
df3_400 = df3[ (df3['대여소번호'] >= 400) & (df3['대여소번호'] < 500) ]

# 대여지점별 누적 이용거리가 가장 큰 지점 찾기
df3_400_dis = df3_400.groupby('대여소번호').sum()['이용거리']
# print(df3_400_dis.sort_values())

# 420이 제일 큼
print(420)

420


### 제2유형
#### 01.

In [233]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action='ignore')

## 00. 데이터 가져오기
X_train = pd.read_csv('../data/모의고사/03회/train_insurance.csv', encoding = 'cp949')
X_test = pd.read_csv('../data/모의고사/03회/test_insurance.csv', encoding = 'cp949')
y_test = pd.read_csv('../data/모의고사/03회/insurance_y_test.csv', encoding = 'cp949')

## 01. 데이터 확인
print("\n--------------------------------------데이터 확인")
# print("\nX_train: ", X_train.shape)    # (1058, 7)
# print("\n", X_train.info())
# print("\n", X_train.head())

# print("\nX_test: ", X_test.shape)      # (280, 6)
# print("\n", X_test.info())
# print("\n", X_test.head())

## 02. y_train 데이터셋 만들기
y_train = X_train['charges'].copy()

# print("\ny_train: ", y_train.shape)    # (1058,)
# print("\n", y_train.info())
# print("\n", y_train.head())

X_train = X_train.drop(columns = 'charges', axis = 1)
print("\nX_train: ", X_train.shape)    # (1058, 6)
print("\n", X_train.info())

## 03. 이상치 확인
# print(X_train.isnull().sum())  # 없음
# print(X_test.isnull().sum())   # 없음


## 03. 명목형 변수의 unique값 먼저 확인 age, sex, smoker, region
print("\n--------------------------------------명목형 데이터 확인")
# print(X_train['age'].value_counts())  # 40S    226, 20S    215, 50S    214, 30S    196, 10S    110, 60S     97
# print(X_train['sex'].value_counts())  # male      538, female    520
# print(X_train['smoker'].value_counts())  # no     844, yes    214
# print(X_train['region'].value_counts())  # southeast    288, northeast    261, southwest    258, northwest    251

# 명목형 변수 따로 저장
X_train_cate = X_train[ ['age', 'sex', 'smoker', 'region'] ]
X_train_cate['age'] = X_train_cate['age'].map( {'10S':0, '20S':1, '30S':2, '40S':3, '50S':4, '60S':5} )
X_train_cate['sex'] = X_train_cate['sex'].map( {'male':0, 'female':1} )
X_train_cate['smoker'] = X_train_cate['smoker'].map( {'no':0, 'yes':1} )
X_train_cate['region'] = X_train_cate['region'].map( {'southeast':0, 'northeast':1, 'southwest':2, 'northwest':3} )

X_test_cate = X_test[ ['age', 'sex', 'smoker', 'region'] ]
X_test_cate['age'] = X_test_cate['age'].map( {'10S':0, '20S':1, '30S':2, '40S':3, '50S':4, '60S':5} )
X_test_cate['sex'] = X_test_cate['sex'].map( {'male':0, 'female':1} )
X_test_cate['smoker'] = X_test_cate['smoker'].map( {'no':0, 'yes':1} )
X_test_cate['region'] = X_test_cate['region'].map( {'southeast':0, 'northeast':1, 'southwest':2, 'northwest':3} )

# 수치형 변수 따로 저장
X_train_conti = X_train[ ['bmi', 'children'] ]
X_test_conti = X_test[ ['bmi', 'children'] ]

# conti와 cate 합치기
X_train = pd.concat( [X_train_cate, X_train_conti], axis = 1 )
X_test = pd.concat( [X_test_cate, X_test_conti], axis = 1 )


## 05. 데이터 분할
print("\n--------------------------------------데이터 분할")
from sklearn.model_selection import train_test_split
X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(X_train,
                                                  y_train,
                                                  random_state = 1234,
                                                  test_size = 0.2)

# shape 확인
print("X_TRAIN: ", X_TRAIN.shape)  # (846, 6)
print("X_VAL: ", X_VAL.shape)      # (212, 6)
print("y_TRAIN: ", y_TRAIN.shape)  # (846,)
print("y_VAL: ", y_VAL.shape)      # (212,)


## 06. 명목형 변수 원핫인코딩
print("\n--------------------------------------명목형 데이터 원핫인코딩")
from sklearn.preprocessing import OneHotEncoder
col_cate = ['age', 'sex', 'smoker', 'region']
X_TRAIN_cate = X_TRAIN[col_cate]
X_VAL_cate = X_VAL[col_cate]
X_test_cate = X_test[col_cate]

encoder = OneHotEncoder(handle_unknown='ignore', sparse = False).fit(X_TRAIN_cate)
X_TRAIN_oh = encoder.transform(X_TRAIN_cate)
X_VAL_oh = encoder.transform(X_VAL_cate)
X_test_oh = encoder.transform(X_test_cate)

## 07. 수치형 변수 스케일링
print("\n--------------------------------------수치형 데이터 스케일링")
from sklearn.preprocessing import StandardScaler
col_conti = ['bmi', 'children']
X_TRAIN_conti = X_TRAIN[col_conti]
X_VAL_conti = X_VAL[col_conti]
X_test_conti = X_test[col_conti]

scaler = StandardScaler().fit(X_TRAIN_conti)
X_TRAIN_std = scaler.transform(X_TRAIN_conti)
X_VAL_std = scaler.transform(X_VAL_conti)
X_test_std = scaler.transform(X_test_conti)


## 08. std와 oh df 합치기
X_TRAIN = np.concatenate( [X_TRAIN_oh, X_TRAIN_std], axis = 1)
X_VAL = np.concatenate( [X_VAL_oh, X_VAL_std], axis = 1)
X_test = np.concatenate( [X_test_oh, X_test_std], axis = 1)

## 09. 입력데이터 준비
print("\n--------------------------------------입력데이터 준비")
y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

## 10. 모델학습 (랜덤포레스트)
print("\n--------------------------------------모델학습")
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 500,
                           max_depth = 5,
                           min_samples_leaf = 10,
                           max_features = 50,
                           random_state=1234)
# print(help(RandomForestRegressor))

model_rf = rf.fit(X_TRAIN, y_TRAIN)

## 11. VAL 데이터로 예측
y_pred = model_rf.predict(X_VAL)

## 12. 평가지표로 확인 (RMSE, MAPE 등)
print("\n--------------------------------------평가지표")
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
mse = mean_squared_error(y_VAL, y_pred, squared = False)
mape = mean_absolute_percentage_error(y_VAL, y_pred)
print("mse: ", mse)
print("mape: ", mape)

# test 데이터로 예측
y_result = model_rf.predict(X_test)


## 12. 결과 제출
obj = {'pred': y_result}
result3 = pd.DataFrame(obj)

# 저장
result3.to_csv('모의고사03회_2유형_결과.csv', index=False)

## 13. y_test 데이터가 있으므로 RMSE, MAPE 확인해보기
y_test = y_test.values.ravel()
mse_result = mean_squared_error(y_test, y_result, squared = False)
mape_result = mean_absolute_percentage_error(y_test, y_result)
print("mse: ", mse_result)
print("mape: ", mape_result)


--------------------------------------데이터 확인

X_train:  (1058, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1058 non-null   object 
 1   sex       1058 non-null   object 
 2   bmi       1058 non-null   float64
 3   children  1058 non-null   int64  
 4   smoker    1058 non-null   object 
 5   region    1058 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 49.7+ KB

 None

--------------------------------------명목형 데이터 확인

--------------------------------------데이터 분할
X_TRAIN:  (846, 6)
X_VAL:  (212, 6)
y_TRAIN:  (846,)
y_VAL:  (212,)

--------------------------------------명목형 데이터 원핫인코딩

--------------------------------------수치형 데이터 스케일링

--------------------------------------입력데이터 준비

--------------------------------------모델학습

--------------------------------------평가지표
mse:  4645.631933454901
mape:  0.3183050